In [1]:

import tensorflow as tf
print(tf.__version__)


2.16.1


In [2]:
pip install --upgrade tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade tensorflow


Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow as tf
print(tf.__version__)


2.16.1


In [5]:
pip install keras


Note: you may need to restart the kernel to use updated packages.


In [6]:
from keras.layers import Input


In [7]:
from tensorflow.keras.layers import Input
import tensorflow as tf
import pathlib
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, LeakyReLU, Activation, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from numpy import zeros, ones
from numpy.random import randint
from matplotlib import pyplot as plt


In [8]:
import numpy as np
import os
from skimage.transform import resize
from matplotlib import pyplot as plt

# Define file paths

input_dir = r"C:\Users\net pc\Desktop\LIVER TUMOR\benign_input"
label_dir = r"C:\Users\net pc\Desktop\LIVER TUMOR\benign_output"


def load_data(input_dir, label_dir, target_shape):
    input_files = os.listdir(input_dir)
    label_files = os.listdir(label_dir)
    
    # Sort files to ensure corresponding inputs and labels match
    input_files.sort()
    label_files.sort()
    
    trainA = []
    trainB = []
    for i in range(len(input_files)):
        input_img = plt.imread(os.path.join(input_dir, input_files[i]))
        label_img = plt.imread(os.path.join(label_dir, label_files[i]))
        
        # Resize images to target shape if they have different shapes
        if input_img.shape != target_shape:
            input_img = resize(input_img, target_shape)
        if label_img.shape != target_shape:
            label_img = resize(label_img, target_shape)
        
        trainA.append(input_img)
        trainB.append(label_img)
    
    return np.array(trainA), np.array(trainB)



# Define target shape for images (e.g., (256, 256, 3))
target_shape = (256, 256, 3)

# Load data
data = load_data(input_dir, label_dir, target_shape)
print("Data loaded successfully!")


Data loaded successfully!


In [9]:
# Define discriminator
def define_discriminator(image_shape):
    init = RandomNormal(stddev=0.02)
    in_src_image = Input(shape=image_shape, name='input_image')
    in_target_image = Input(shape=image_shape, name='target_image')
    merged = Concatenate()([in_src_image, in_target_image])
    
    # Discriminator architecture
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    patch_out = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(patch_out)
    
    model = Model([in_src_image, in_target_image], patch_out)
    model.compile(loss='binary_crossentropy', optimizer='adam')
   
    return model


In [10]:
def define_generator(image_shape=(256, 256, 3)):
    init = RandomNormal(stddev=0.02)
    in_image = Input(shape=image_shape)
    
    # Encoder
    e1 = Conv2D(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(in_image)
    e2 = Conv2D(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e1))
    e2 = BatchNormalization()(e2)
    e3 = Conv2D(256, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e2))
    e3 = BatchNormalization()(e3)
    e4 = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e3))
    e4 = BatchNormalization()(e4)
    e5 = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e4))
    e5 = BatchNormalization()(e5)
    e6 = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e5))
    e6 = BatchNormalization()(e6)
    e7 = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e6))
    e7 = BatchNormalization()(e7)
    
    # Bottleneck
    b = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(LeakyReLU(alpha=0.2)(e7))
    b = Activation('relu')(b)
    
    # Decoder
    d1 = Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(Activation('relu')(b))
    d1 = BatchNormalization()(d1)
    d1 = Dropout(0.5)(d1)
    d1 = Concatenate()([d1, e7])
    d2 = Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(Activation('relu')(d1))
    d2 = BatchNormalization()(d2)
    d2 = Dropout(0.5)(d2)
    d2 = Concatenate()([d2, e6])
    d3 = Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(Activation('relu')(d2))
    d3 = BatchNormalization()(d3)
    d3 = Concatenate()([d3, e5])
    d4 = Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(Activation('relu')(d3))
    d4 = BatchNormalization()(d4)
    d4 = Concatenate()([d4, e4])
    d5 = Conv2DTranspose(256, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(Activation('relu')(d4))
    d5 = BatchNormalization()(d5)
    d5 = Concatenate()([d5, e3])
    d6 = Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(Activation('relu')(d5))
    d6 = BatchNormalization()(d6)
    d6 = Concatenate()([d6, e2])
    d7 = Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(Activation('relu')(d6))
    d7 = BatchNormalization()(d7)
    d7 = Concatenate()([d7, e1])
    
    # Output
    g = Conv2DTranspose(image_shape[2], (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d7)
    out_image = Activation('tanh')(g)
    
    model = Model(in_image, out_image)
    return model


In [11]:
def generate_real_samples(dataset, n_samples, patch_shape):
    # unpack dataset
    trainA, trainB = dataset
    # choose random instances
    ix = randint(0, trainA.shape[0], n_samples)
    # retrieve selected images
    X1, X2 = trainA[ix], trainB[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return [X1, X2], y


In [12]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y


In [13]:
def define_gan(g_model, d_model, image_shape):
    # Make weights in the discriminator not trainable
    d_model.trainable = False
    
    # Input tensor for the generator model
    gen_input = Input(shape=image_shape)
    
    # Connect the generator output to the discriminator input
    gan_output = d_model([gen_input, g_model(gen_input)])
    
    # Define GAN model taking generator input and outputting discriminator decision
    gan_model = Model(gen_input, gan_output)
    
    # Compile GAN model
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    gan_model.compile(loss=['binary_crossentropy'], optimizer=opt)
    
    return gan_model

# Define discriminator model
d_model = define_discriminator(target_shape)

# Define generator model
g_model = define_generator(target_shape)

# Define GAN model
gan_model = define_gan(g_model, d_model, target_shape)

C:\Users\Public\Efendim\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [14]:

import numpy as np

def train(d_model, g_model, gan_model, dataset, n_epochs=1000, n_batch=1):
    # Determine the output square shape of the discriminator
    n_patch = d_model.output_shape[1]
    # Unpack dataset
    trainA, trainB = dataset
    # Calculate the number of batches per training epoch
    bat_per_epo = int(len(trainA) / n_batch)
    # Calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    
    for i in range(n_steps):
        # Select a batch of real samples
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
        # Generate a batch of fake samples
        X_fakeB, _ = generate_fake_samples(g_model, X_realA, n_patch)
        # Update discriminator for real samples
        d_loss1 = d_model.train_on_batch([X_realA, X_realB], np.ones((n_batch, n_patch, n_patch, 1)))
        # Update discriminator for generated samples
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], np.zeros((n_batch, n_patch, n_patch, 1)))
        # Update the generator
        g_loss = gan_model.train_on_batch(X_realA, [np.ones((n_batch, n_patch, n_patch, 1)), X_realB])
        # Summarize performance
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
        # Summarize model performance
        if (i+1) % (bat_per_epo * 10) == 0:
            summarize_performance(i, g_model, dataset)


In [15]:

# generate samples and save as a plot and save the model
#GAN models do not converge, we just want to find a good balance between
#the generator and the discriminator. Therefore, it makes sense to periodically
#save the generator model and check how good the generated image looks. 
import os
from PIL import Image

from PIL import Image


def save_img(filename, image):
    """Save the image to the specified filename."""
    # Scale the image back to the range [0, 255] before saving
    scaled_image = (image * 255).astype(np.uint8)
    # Convert to PIL Image
    im = Image.fromarray(scaled_image)
    # Save the image
    im.save(filename)
    
# Summarize performance
def summarize_performance(step, g_model, dataset, n_samples=3):
    print("Saving images...")
    # Select a sample of input images
    [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
    # Generate a batch of fake samples
    X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
    # Scale all pixels from [-1,1] to [0,1]
    X_realA = (X_realA + 1) / 2.0
    X_realB = (X_realB + 1) / 2.0
    X_fakeB = (X_fakeB + 1) / 2.0
    

    output_path = "C:\\Users\\net pc\\Desktop\\LIVER TUMOR\\benign_generated2"



    for i in range(n_samples):
        # Save real source image as JPEG
        real_source_image = X_realA[i]
        real_source_filename = os.path.join(output_path, f'real_source_{step}_{i + 1}.jpeg')
        save_img(real_source_filename, real_source_image)

        # Save generated target image as JPEG
        generated_target_image = X_fakeB[i]
        generated_target_filename = os.path.join(output_path, f'generated_target_{step}_{i + 1}.jpeg')
        save_img(generated_target_filename, generated_target_image)

        # Save real target image as JPEG
        real_target_image = X_realB[i]
        real_target_filename = os.path.join(output_path, f'real_target_{step}_{i + 1}.jpeg')
        save_img(real_target_filename, real_target_image)

    # Save the generator model
    model_filename = os.path.join(output_path, f'model_{step}.h5')
    g_model.save(model_filename)

    print(f"Images and model saved at {output_path}")

    


In [ ]:


# Train Pix2Pix models using the GAN model
train(d_model, g_model, gan_model, data, n_epochs=1000, n_batch=1)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step


C:\Users\Public\Efendim\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:74: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


>1, d1[0.633] d2[0.678] g[0.664]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
>2, d1[0.663] d2[0.679] g[0.639]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
>3, d1[0.669] d2[0.678] g[0.621]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
>4, d1[0.672] d2[0.679] g[0.604]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>5, d1[0.674] d2[0.679] g[0.588]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
>6, d1[0.675] d2[0.679] g[0.575]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
>7, d1[0.676] d2[0.679] g[0.562]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
>8, d1[0.676] d2[0.680] g[0.551]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
>9, d1[0.677] d2[0.680] g[0.542]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
>10, d1[0.677] d2[0.680] g[0.533]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
>11, d1[0.678] d2[0.680] g[0.525]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
>12, d1[0.678] d2[0.680] g[0.517]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
>13, d1[0.678] d2[0.681] g[0.511]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
>14, d1[0.679] d2[0.681] g[0.504]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
>

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>65, d1[0.700] d2[0.701] g[0.384]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>66, d1[0.701] d2[0.702] g[0.383]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
>67, d1[0.701] d2[0.702] g[0.382]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
>68, d1[0.702] d2[0.703] g[0.381]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
>69, d1[0.702] d2[0.703] g[0.380]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
>70, d1[0.703] d2[0.704] g[0.379]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
>71, d1[0.703] d2[0.704] g[0.379]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
>72, d1[0.704] d2[0.705] g[0.378]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
>73, d1[0.704] d2[0.705] g[0.377]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
>74, d1[0.705] d2[0.706] g[0.376]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
>75, d1[0.705] d2[0.706] g[0.375]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
>76, d1[0.706] d2[0.707] g[0.375]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
>77, d1[0.706] d2[0.707] g[0.374]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
>78, d1[0.707] d2[0.708] g

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
>142, d1[0.743] d2[0.744] g[0.342]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
>143, d1[0.744] d2[0.744] g[0.341]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
>144, d1[0.744] d2[0.745] g[0.341]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
>145, d1[0.745] d2[0.746] g[0.341]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
>146, d1[0.745] d2[0.746] g[0.340]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
>147, d1[0.746] d2[0.747] g[0.340]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
>148, d1[0.747] d2[0.748] g[0.340]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
>149, d1[0.747] d2[0.748] g[0.339]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
>150, d1[0.748] d2[0.749] g[0.339]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
>151, d1[0.748] d2[0.749] g[0.339]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
>152, d1[0.749] d2[0.750] g[0.338]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
>153, d1[0.750] d2[0.751] g[0.338]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
>154, d1[0.750] d2[0.751] g[0.338]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
>155, d1[0.

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 410ms/step
>902, d1[0.979] d2[0.979] g[0.279]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 397ms/step
>903, d1[0.979] d2[0.979] g[0.279]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 450ms/step
>904, d1[0.979] d2[0.979] g[0.279]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 428ms/step
>905, d1[0.979] d2[0.980] g[0.279]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 483ms/step
>906, d1[0.979] d2[0.980] g[0.279]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step
>907, d1[0.979] d2[0.980] g[0.279]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 408ms/step
>908, d1[0.980] d2[0.980] g[0.279]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 403ms/step
>909, d1[0.980] d2[0.980] g[0.279]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 519ms/step
>910, d1[0.980] d2[0.980] g[0.279]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 495ms/step
>911, d1[0.980] d2[0.980] g[0.279]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 391ms/step
>912, d1[0.980] d2[0.980] g[0.279]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 441ms/step
>913, d1[0.980] d2[0.980] g[0.279]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 409ms/step
>914, d1[0.980] d2[0.981] g[0.279]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 419ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 603ms/step
>978, d1[0.986] d2[0.987] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step
>979, d1[0.987] d2[0.987] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 525ms/step
>980, d1[0.987] d2[0.987] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 467ms/step
>981, d1[0.987] d2[0.987] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 451ms/step
>982, d1[0.987] d2[0.987] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step
>983, d1[0.987] d2[0.987] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step
>984, d1[0.987] d2[0.987] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 660ms/step
>985, d1[0.987] d2[0.987] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 714ms/step
>986, d1[0.987] d2[0.987] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 457ms/step
>987, d1[0.987] d2[0.988] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 528ms/step
>988, d1[0.987] d2[0.988] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 554ms/step
>989, d1[0.987] d2[0.988] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step
>990, d1[0.988] d2[0.988] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 479ms/step

Images and model saved at C:\Users\net pc\Desktop\LIVER TUMOR\benign_generated2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 422ms/step
>1001, d1[0.989] d2[0.989] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 567ms/step
>1002, d1[0.989] d2[0.989] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 415ms/step
>1003, d1[0.989] d2[0.989] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 505ms/step
>1004, d1[0.989] d2[0.989] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 486ms/step
>1005, d1[0.989] d2[0.989] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 463ms/step
>1006, d1[0.989] d2[0.989] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 571ms/step
>1007, d1[0.989] d2[0.989] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 615ms/step
>1008, d1[0.989] d2[0.989] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 478ms/step
>1009, d1[0.989] d2[0.989] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 482ms/step
>1010, d1[0.989] d2[0.990] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 692ms/step
>1011, d1[0.989] d2[0.990] g[0.277]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 635ms/step
>1012, d1[0.989] d2[0.990] g[0.277]
1/1 ━━━━━━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 506ms/step
>1075, d1[0.995] d2[0.995] g[0.276]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 484ms/step
>1076, d1[0.995] d2[0.995] g[0.276]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 500ms/step
>1077, d1[0.995] d2[0.995] g[0.276]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 484ms/step
>1078, d1[0.995] d2[0.995] g[0.275]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 506ms/step
>1079, d1[0.995] d2[0.995] g[0.275]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 484ms/step
>1080, d1[0.995] d2[0.995] g[0.275]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 516ms/step
>1081, d1[0.995] d2[0.995] g[0.275]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 618ms/step
>1082, d1[0.995] d2[0.995] g[0.275]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 625ms/step
>1083, d1[0.995] d2[0.995] g[0.275]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 547ms/step
>1084, d1[0.995] d2[0.995] g[0.275]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 525ms/step
>1085, d1[0.995] d2[0.996] g[0.275]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 551ms/step
>1086, d1[0.995] d2[0.996] g[0.275]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 542ms/step
>1087, d1[0.995] d2[0.996] g[0.275]
1/1 ━━━━━━━━━━━━━━━━━━━━ 

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 562ms/step
>1150, d1[1.000] d2[1.000] g[0.274]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 578ms/step
>1151, d1[1.000] d2[1.000] g[0.274]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 609ms/step
>1152, d1[1.000] d2[1.000] g[0.274]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 562ms/step
>1153, d1[1.000] d2[1.000] g[0.274]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 586ms/step
>1154, d1[1.000] d2[1.000] g[0.274]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 551ms/step
>1155, d1[1.000] d2[1.000] g[0.274]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 609ms/step
>1156, d1[1.000] d2[1.001] g[0.274]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 578ms/step
>1157, d1[1.000] d2[1.001] g[0.274]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 641ms/step
>1158, d1[1.000] d2[1.001] g[0.274]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 641ms/step
>1159, d1[1.001] d2[1.001] g[0.274]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 562ms/step
>1160, d1[1.001] d2[1.001] g[0.274]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 562ms/step
>1161, d1[1.001] d2[1.001] g[0.274]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 547ms/step
>1162, d1[1.001] d2[1.001] g[0.274]
1/1 ━━━━━━━━━━━━━━━━━━━━ 